In [424]:
import sys  
import time  
import math  
import uuid  
import numpy as np  
import matplotlib.pyplot as plt
import pickle
import plotly.graph_objs as go
from numpy import errstate  
from sklearn.datasets import fetch_openml  
from sklearn.model_selection import train_test_split  
from sklearn.preprocessing import OneHotEncoder  
from tqdm import tqdm

In [425]:
sys.setrecursionlimit(10000)  

In [426]:
class VarValue:
    def __init__(self, value, children=(), varname=''):
        self.varname = varname
        self.value = value
        self.derivative_to = {}
        self.children = children

    def __chain_rule(self, dSelfdx, child):
        if(child.varname[:5] != 'const'):
            for grandchild_varname in child.derivative_to:
                if(grandchild_varname != 'const'):
                    if(grandchild_varname in self.derivative_to):

                        self.derivative_to[grandchild_varname] += dSelfdx * child.derivative_to[grandchild_varname]
                    else:
                        self.derivative_to[grandchild_varname] = dSelfdx * child.derivative_to[grandchild_varname]
            if(len(self.derivative_to) == 0):
              raise ValueError(self.varname, child.varname)

    def relu(self):
        out = VarValue(max(0,self.value), children=(self,), varname='out_relu_'+str(uuid.uuid4()))
        if(self.varname[:5] != 'const'):
            dodx = 0 if self.value <= 0 else 1
            if(len(self.children) == 0):
                out.derivative_to[self.varname] = dodx
            else:
                out.__chain_rule(dodx, self)
        return out

    def ln(self):
        out = VarValue(math.log(self.value), children=(self,), varname='out_ln_'+str(uuid.uuid4()))
        if(self.varname[:5] != 'const'):
            dodx = 1/(self.value)
            if(len(self.children) == 0):
                out.derivative_to[self.varname] = dodx
            else:
                out.__chain_rule(dodx, self)
        return out

    def __mul__(self, other):
        other = other if isinstance(other, VarValue) else VarValue(other, varname='const'+str(uuid.uuid4()))
        out = VarValue(self.value * other.value, children=(self, other), varname='out_mul_'+str(uuid.uuid4()))

        if(self.varname[:5] != 'const'):
            dodx1 = other.value
            if(len(self.children) == 0):
                out.derivative_to[self.varname] = dodx1
            else:
                out.__chain_rule(dodx1, self)
        if(other.varname[:5] != 'const'):
            dodx2 = self.value
            if(len(other.children) == 0):
                out.derivative_to[other.varname] = dodx2
            else:
                out.__chain_rule(dodx2, other)
        return out

    def __add__(self, other):
        other = other if isinstance(other, VarValue) else VarValue(other, varname='const'+str(uuid.uuid4()))
        out = VarValue(self.value + other.value, children=(self, other), varname='out_add_'+str(uuid.uuid4()))
        if(self.varname[:5] != 'const'):
            dodx1 = 1
            if(len(self.children) == 0):
                out.derivative_to[self.varname] = dodx1
            else:
                out.__chain_rule(dodx1, self)
        if(other.varname[:5] != 'const'):
            dodx2 = 1
            if(len(other.children) == 0):
                out.derivative_to[other.varname] = dodx2
            else:
                out.__chain_rule(dodx2, other)
        return out

    def __sub__(self, other):
        return self + (-other)

    def __truediv__(self, other):
        return self * other**-1

    def __neg__(self):
        return self * -1

    def __pow__(self, other):
        other = other if isinstance(other, VarValue) else VarValue(other, varname='const'+str(uuid.uuid4()))
        try:
            with errstate(over='raise', invalid='raise'):
                result = float(self.value) ** float(other.value)
        except (OverflowError, FloatingPointError):
            if abs(float(self.value)) > 1:
                result = math.inf
            else:
                result = 0

        out = VarValue(result, children=(self, other), varname='out_pow_'+str(uuid.uuid4()))

        if(self.varname[:5] != 'const'):
            dodx = other.value * self.value**(other.value-1)
            if(len(self.children) == 0):
                out.derivative_to[self.varname] = dodx
            else:
                out.__chain_rule(dodx, self)
        return out

    def __rmul__(self, other):
        return self * other

    def __radd__(self, other):
        return self + other

    def __rsub__(self, other):
        return other + (-self)

    def __rpow__(self, other):
        other = other if isinstance(other, VarValue) else VarValue(other, varname='const'+str(uuid.uuid4()))
        out = VarValue(other.value**self.value, children=(self, other), varname='out_rpow_'+str(uuid.uuid4()))
        if self.varname:
            dodx = other.value**self.value * math.log(other.value)
            if(len(self.children) == 0):
                out.derivative_to[self.varname] = dodx
            else:
                out.__chain_rule(dodx, self)
        return out

    def __rtruediv__(self, other):
        return other * self**-1

    # Equality
    def __eq__(self, other):
        if isinstance(other, VarValue):
            return self.varname == other.varname
        return self.varname == other

    # Inequality
    def __ne__(self, other):
        return not self.__eq__(other)

    # Less than
    def __lt__(self, other):
        if isinstance(other, VarValue):
            return self.value < other.value
        return self.value < other

    # Less than or equal
    def __le__(self, other):
        if isinstance(other, VarValue):
            return self.value <= other.value
        return self.value <= other

    # Greater than
    def __gt__(self, other):
        if isinstance(other, VarValue):
            return self.value > other.value
        return self.value > other

    # Greater than or equal
    def __ge__(self, other):
        if isinstance(other, VarValue):
            return self.value >= other.value
        return self.value >= other
    
    def log(self):
        out = VarValue(math.log(self.value), children=(self,), varname='outlog'+str(uuid.uuid4()))
        if self.varname[:5] != 'const':
            dodx = 1 / self.value
            if len(self.children) == 0:
                out.derivative_to[self.varname] = dodx
            else:
                out.__chain_rule(dodx, self)
        return out

    def exp(self):
        out = VarValue(math.exp(self.value), children=(self,), varname='out_exp_'+str(uuid.uuid4()))
        if self.varname[:5] != 'const':
            dodx = math.exp(self.value)
            if len(self.children) == 0:
                out.derivative_to[self.varname] = dodx
            else:
                out.__chain_rule(dodx, self)
        return out



In [427]:
class Layer:
    def __init__(self,n_neurons=3, init='zero', activation='relu', weights=None, biases=None):
        # weights & biases ditambah di init buat memfasilitasi loading weight & bias
        self.n_neurons = n_neurons
        self.current_input_batch = None
        self.init = init    # zero/uniform/normal/xavier/he - harusnya gaperlu disini, ini di layer langsung harusnya
        self.weights = weights
        self.biases = biases
        self.activation = activation    # linear/relu/sigmoid/tanh/softmax/binary_step/leaky_relu/prelu/elu/swish(ini)/gelu(ini)
        self.learning_rate = None

        self.net = None
        self.out = None

    def __update_weights_dEdW(self, dEdW):
        clip_value = 1.0  # maksimal update yang diperbolehkan
        dEdW = np.clip(dEdW, -clip_value, clip_value)
        self.weights -= self.learning_rate * dEdW

    def __update_biases_dEdB(self, dEdB):
        clip_value = 1.0  # maksimal update yang diperbolehkan
        dEdB = np.clip(dEdB, -clip_value, clip_value)
        self.biases -= self.learning_rate * dEdB

    def __update_weights_err_term(self, err_term):
        for input in self.current_input_batch:
            for i in self.weights:
                for j in i:
                    self.weights += self.learning_rate*err_term[j]*self.input[i]

    def __update_biases_err_term(self, err_term):
        for i in self.weights:
            for j in i:
                self.weights += self.learning_rate*err_term[j]*1

    def forward(self, current_input_batch):
        self.current_input_batch = current_input_batch

        if(self.weights is None):
            
            if(self.init == 'zero'):
                self.weights = np.array([[VarValue(0,varname='w_'+str(uuid.uuid4())) for _ in range(self.n_neurons)] for _ in range(len(self.current_input_batch[0]))])
                self.biases = np.array([VarValue(0,varname='b_'+str(uuid.uuid4())) for _ in range(self.n_neurons)])
                print("TESTT")
                print(self.weights)
                print(self.biases)

            # Ini semua diround soalnya hasil operasinya kegedean, kena warning wkwkwk. CMIIW yh harusnya berapa angka di belakang koma - @evelynnn04

            elif(self.init == 'uniform'):
                limit = 1 / np.sqrt(len(self.current_input_batch[0]))
                self.weights = np.array([[VarValue(round(np.random.uniform(-limit, limit), 4), varname='w_'+str(uuid.uuid4())) 
                                        for _ in range(self.n_neurons)] for _ in range(len(self.current_input_batch[0]))])
                self.biases = np.array([VarValue(round(np.random.uniform(-limit, limit), 4), varname='b_'+str(uuid.uuid4())) 
                                        for _ in range(self.n_neurons)])

            elif self.init == 'normal':
                std = 1 / np.sqrt(len(self.current_input_batch[0]))
                self.weights = np.array([[VarValue(round(np.random.normal(0, std), 4), varname='w_'+str(uuid.uuid4())) 
                                        for _ in range(self.n_neurons)] for _ in range(len(self.current_input_batch[0]))])
                self.biases = np.array([VarValue(round(np.random.normal(0, std), 4), varname='b_'+str(uuid.uuid4())) 
                                        for _ in range(self.n_neurons)])

            elif self.init == 'xavier':
                std = np.sqrt(2 / (len(self.current_input_batch[0]) + self.n_neurons))
                self.weights = np.array([[VarValue(round(np.random.normal(0, std), 4), varname='w_'+str(uuid.uuid4())) 
                                        for _ in range(self.n_neurons)] for _ in range(len(self.current_input_batch[0]))])
                self.biases = np.array([VarValue(round(0, 4), varname='b_'+str(uuid.uuid4())) 
                                        for _ in range(self.n_neurons)])

            elif self.init == 'he':
                std = np.sqrt(2 / len(self.current_input_batch[0]))
                self.weights = np.array([[VarValue(round(np.random.normal(0, std), 4), varname='w_'+str(uuid.uuid4())) 
                                        for _ in range(self.n_neurons)] for _ in range(len(self.current_input_batch[0]))])
                self.biases = np.array([VarValue(round(0, 4), varname='b_'+str(uuid.uuid4())) 
                                        for _ in range(self.n_neurons)])


        self.net = (np.dot(self.current_input_batch, self.weights)) + self.biases

        if(self.activation == 'linear'):
            self.out = self.net

        elif(self.activation == 'relu'):
            self.out = np.array([[net.relu() for net in row] for row in self.net])

        elif(self.activation == 'sigmoid'):
            for i in self.net:
                for j in i:
                    j.value = np.clip(j.value, -500, 500)
            self.out = 1 / (1 + (math.exp(1))**(-self.net))

        elif(self.activation == 'tanh'):
            self.out = (math.exp(1)**self.net - math.exp(1)**(-self.net))/(math.exp(1)**self.net + math.exp(1)**(-self.net))

        elif self.activation == 'softmax':
            exp_values = np.array([[net.exp() for net in row] for row in self.net])
            sums = np.array([[VarValue(sum(n.value for n in row), varname='sum_exp_'+str(uuid.uuid4()))] for row in exp_values])
            self.out = np.array([[exp_values[i][j] / sums[i][0] for j in range(len(exp_values[i]))] for i in range(len(exp_values))])


    def backward(self, err=None):
        if err is None:
            raise ValueError("err (VarValue object) harus diberikan dari fungsi loss sebagai parameter!")

        # Ambil derivatif langsung dari err (loss)
        dEdW = np.zeros((len(self.weights), len(self.weights[0])))
        dEdB = np.zeros(len(self.biases))

        for i in range(len(self.weights)):
            for j in range(len(self.weights[i])):
                varname = self.weights[i][j].varname
                dEdW[i][j] = err.derivative_to.get(varname, 0.0)

        for j in range(len(self.biases)):
            varname = self.biases[j].varname
            dEdB[j] = err.derivative_to.get(varname, 0.0)

        # Update bobot dan bias
        clip_val = 1.0
        self.weights -= self.learning_rate * np.clip(dEdW, -clip_val, clip_val)
        self.biases -= self.learning_rate * np.clip(dEdB, -clip_val, clip_val)


    def clean_derivative(self):
        for input in self.current_input_batch:
            for x in input:
                x.derivative_to.clear()
                x.children = ()

        for i in self.weights:
            for j in i:
                j.derivative_to.clear()
                j.children = ()

        for b in self.biases:
            b.derivative_to ={}
            b.children = ()

        for i in self.net:
            for j in i:
                j.derivative_to.clear()
                j.children = ()

        for i in self.out:
            for j in i:
                j.derivative_to.clear()
                j.children = ()

In [ ]:
class FFNN:
    def __init__(self, loss='mse', batch_size=1, learning_rate=0.01, epochs=20, verbose=0):
        self.loss = loss    # mse/bce/cce
        self.batch_size=batch_size
        self.learning_rate=learning_rate
        self.epochs=epochs
        self.verbose=verbose
        self.layers = None
        self.weights = []
        self.bias = []
        self.x = None
        self.y = None
        self.onehot_encoder = OneHotEncoder(categories='auto')

    def __loss(self, out, target):
        if out.shape != target.shape:
            print("Output shape: ", out.shape)
            print("Target shape: ", target.shape)
            raise ValueError("Shape not match")

        if self.loss == 'mse':
            mse = (1/target.shape[1]) * np.square(target - out)
            return np.sum(mse)

        elif self.loss == 'cce':
            epsilon = 1e-15
            loss = VarValue(0.0, varname='loss_const')

            for i in range(target.shape[0]):            # loop untuk batch
                for j in range(target.shape[1]):        # loop untuk neuron output
                    target_val = VarValue(target[i, j], varname=f'target_{i}_{j}')
                    out[i, j].value = np.clip(out[i, j].value, epsilon, 1 - epsilon)
                    loss = loss + (target_val * out[i, j].log())

            loss = loss * (-1 / self.batch_size)
            return loss

    def build_layers(self, *layers: Layer):
        self.layers = layers
        for layer in self.layers:
            layer.learning_rate = self.learning_rate

    def fit(self, x, y):
        self.x = x
        self.y = self.onehot_encoder.fit_transform(y.reshape(-1, 1)).toarray()
        total_batch = (len(x)+self.batch_size-1)//self.batch_size
        start_global = time.time()

        epoch_iterator = tqdm(range(self.epochs), desc="Training Progress", unit="epoch") if self.verbose else range(self.epochs)

        for epoch in epoch_iterator:
            if self.verbose:
                print(f"Epoch {epoch+1}")

            batch_iterator = tqdm(range(total_batch), desc=f"Epoch {epoch + 1}", unit="batch", leave=False) if self.verbose else range(total_batch)

            for i in batch_iterator:
                # Ambil batch
                x_batch = self.x[i*self.batch_size:(i+1)*self.batch_size] if (i+1) < total_batch else self.x[i*self.batch_size:]
                y_batch = self.y[i*self.batch_size:(i+1)*self.batch_size] if (i+1) < total_batch else self.y[i*self.batch_size:]

                # Forward pass
                batch_input = x_batch
                for layer in self.layers:
                    layer.forward(batch_input)
                    batch_input = layer.out
                out = batch_input

                # Hitung loss (VarValue object)
                error = self.__loss(out, y_batch)

                # Backward pass
                for layer in reversed(self.layers):
                    layer.backward(err=error)

                for layer in self.layers:
                    layer.clean_derivative()

        end_global = time.time()
        print("Total Duration:", end_global-start_global)

    def predict(self, x_predict):
        batch_input = x_predict
        for layer in self.layers:
            layer.forward(batch_input)
            batch_input = layer.out
        out = batch_input
        return out

    def visualize(self):
        num_layers = len(self.layers)
        
        # Color palette
        layer_colors = {
            0: 'yellow',    # Input layer
            -1: 'salmon',   # Output layer
            'hidden': 'lightblue'  # Hidden layers
        }
        
        nodes_x = []
        nodes_y = []
        node_colors = []
        node_texts = []
        
        edges_x = []
        edges_y = []
        edge_texts = []
        
        for layer_idx, layer in enumerate(self.layers):
            if num_layers == 1:
                color = layer_colors[-1]
            elif layer_idx == 0:
                color = layer_colors[0]
            elif layer_idx == num_layers - 1:
                color = layer_colors[-1]
            else:
                color = layer_colors['hidden']
            
            n_neurons = layer.n_neurons
            y_positions = np.linspace(0, 1, n_neurons)
            x_pos = layer_idx / (num_layers - 1) if num_layers > 1 else 0.5
            
            for neuron_idx, y_pos in enumerate(y_positions):
                nodes_x.append(x_pos)
                nodes_y.append(y_pos)
                node_colors.append(color)
                node_texts.append(f"Layer {layer_idx}, Neuron {neuron_idx}<br>Activation: {layer.activation}")
            
            if layer_idx < num_layers - 1:
                next_layer = self.layers[layer_idx + 1]
                next_x = (layer_idx + 1) / (num_layers - 1) if num_layers > 1 else 0.5
                next_n_neurons = next_layer.n_neurons
                next_y_positions = np.linspace(0, 1, next_n_neurons)
                
                for curr_neuron_idx, curr_y in enumerate(y_positions):
                    for next_neuron_idx, next_y in enumerate(next_y_positions):
 
                        edges_x.extend([x_pos, next_x, None])
                        edges_y.extend([curr_y, next_y, None])
                        
                        weight_text = "Weight: Not initialized"
                        if layer.weights is not None:
                            try:
                                weight = layer.weights[curr_neuron_idx][next_neuron_idx]
                                weight_text = f"Weight: {weight}"
                            except:
                                weight_text = "Weight: Unavailable"
                        
                        edge_texts.append(weight_text)
        
        # Create nodes
        node_trace = go.Scatter(
            x=nodes_x, y=nodes_y,
            mode='markers',
            hoverinfo='text',
            marker=dict(
                showscale=False,
                color=node_colors,
                size=15,
                line_width=2
            ),
            text=node_texts
        )
        
        # Create edges
        edge_trace = go.Scatter(
            x=edges_x, y=edges_y,
            mode='lines',
            line=dict(width=0.5, color='#888'),
            hoverinfo='text',
            text=edge_texts
        )
        
        fig = go.Figure(data=[edge_trace, node_trace],
                        layout=go.Layout(
                            title='Neural Network Architecture',
                            showlegend=False,
                            hovermode='closest',
                            margin=dict(b=0,l=0,r=0,t=40),
                            xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                            yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
                        ))
        
        fig.update_layout(plot_bgcolor='rgba(255,255,255,255)')
        fig.show()

    
    def save(self, filename):
        model_data = {
            'loss': self.loss,
            'batch_size': self.batch_size,
            'learning_rate': self.learning_rate,
            'epochs': self.epochs,
            'layers': [
                {'n_neurons': layer.n_neurons, 'activation': layer.activation, 'weights': layer.weights.tolist(), 'biases': layer.biases.tolist()}
                for layer in self.layers
            ]
        }
        with open(filename, 'wb') as f:
            pickle.dump(model_data, f)
        print(f"Model saved to {filename}")

    def load(self, filename):
        with open(filename, 'rb') as f:
            model_data = pickle.load(f)
        self.loss = model_data['loss']
        self.batch_size = model_data['batch_size']
        self.learning_rate = model_data['learning_rate']
        self.epochs = model_data['epochs']
        self.layers = [
            Layer(n_neurons=layer['n_neurons'], activation=layer['activation'], weights=np.array(layer['weights']), biases=np.array(layer['biases']))
            for layer in model_data['layers']
        ]
        print(f"Model loaded from {filename}")


In [429]:
X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)
y = y.astype(np.uint8)

In [430]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=65500,
    train_size=10,
    random_state=42,
    stratify=y
)

In [431]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [432]:
X_train = np.array([[VarValue(x, varname='x_'+str(uuid.uuid4())) for x in row] for row in X_train])

In [433]:
model_ffnn = FFNN(
    loss='mse',
    batch_size=1,
    learning_rate=0.1,
    epochs=2,
    verbose=1,
)

In [434]:
model_ffnn.build_layers(
    Layer(n_neurons=4, init='uniform', activation='relu'),
    Layer(n_neurons=3, init='uniform', activation='relu'),
    Layer(n_neurons=2, init='uniform', activation='linear'),
    Layer(n_neurons=10, init='uniform', activation='softmax')
)

In [435]:
model_ffnn.fit(X_train, y_train)

Training Progress:   0%|          | 0/2 [00:00<?, ?epoch/s]

Epoch 1


Training Progress:  50%|█████     | 1/2 [00:07<00:07,  7.73s/epoch]

Epoch 2


Training Progress: 100%|██████████| 2/2 [00:16<00:00,  8.38s/epoch]

Total Duration: 16.76672887802124


In [436]:
model_ffnn.visualize()

In [437]:
for layer in model_ffnn.layers:
    print("=====Layer======")
    for i in layer.weights:
        print("==Neuron==")
        for j in i:
            print(j.varname)

=====Layer======
==Neuron==
out_add_3a7e35b0-8f2a-4e7b-9dca-fc0c97d9a790
out_add_89048577-7e31-462b-b190-66a52e6aa5b8
out_add_cd663aa7-5f27-4d5e-9156-03e162b11a76
out_add_23e384c7-45dc-4386-b806-07bdd0d82406
==Neuron==
out_add_b8734a9c-cd08-43d6-a790-0248d1b81651
out_add_c4a73191-8f41-4932-b416-c12859401e35
out_add_f128a53d-d272-48ba-afcf-872bdec0c53c
out_add_053a31ef-8676-422a-b993-d892a85a5a29
==Neuron==
out_add_45b4a7f2-770f-4558-add5-e937850f8b47
out_add_f8ede85a-eb2b-470b-be05-8b64b52d8a72
out_add_c200f698-8d9f-474f-8eae-cf218dd51fa5
out_add_4a003d8b-abcf-43cf-981b-968c5eb5c57c
==Neuron==
out_add_db8cbeb4-a410-4746-8547-069c6855072d
out_add_acbf0ba9-7aaf-46fc-9c82-c78360c4b60f
out_add_e6322bb6-57e3-49b6-8539-9fed29f93582
out_add_634f7266-8065-4f28-a844-c9aea14ea859
==Neuron==
out_add_6fe7dd29-c125-4e41-8c0e-a78d4094aaed
out_add_87612240-0292-4bd7-810a-2c745a8976d9
out_add_fe841ebd-85c2-4029-8518-5c32327b870b
out_add_fc049e0c-f9e4-46c4-954b-7c7911a2c4a7
==Neuron==
out_add_c1b8b04b-

In [438]:
for layer in model_ffnn.layers:
    for i in layer.weights:
        for j in i:
            print(j.value)

-0.0267
0.0215
-0.0139
-0.0007
-0.0193
-0.0311
0.0273
0.0262
-0.0284
-0.0044
-0.0002
0.0035
0.0142
0.0005
-0.0331
-0.0251
0.0147
-0.0064
0.0258
0.0335
-0.0154
-0.0173
-0.0204
0.0092
0.0173
-0.0175
0.0289
-0.0093
0.0289
-0.0076
-0.0146
-0.0238
0.0347
0.0133
-0.0006
0.0121
0.0328
-0.016
-0.0262
0.0106
-0.032
0.0168
0.0253
0.019
-0.0129
-0.0248
0.0246
-0.022
0.0319
-0.0077
0.0208
0.0158
-0.0085
-0.0278
0.0086
0.0195
-0.0352
0.0121
0.0285
0.0014
-0.0132
-0.0237
-0.0172
0.0245
-0.0053
0.0025
-0.0121
-0.0302
0.0352
-0.0001
0.02
-0.0263
-0.0235
0.0077
-0.0118
0.0125
-0.0145
-0.0336
0.0175
-0.0252
-0.0269
0.0163
0.0174
-0.0242
0.0006
-0.0094
-0.0171
0.0175
0.0234
-0.0174
-0.0122
0.002
0.0032
-0.0054
0.0347
-0.0134
0.0344
-0.0342
0.0111
-0.002
0.0236
-0.0177
0.0141
-0.0052
-0.0235
-0.0256
-0.03
-0.0027
0.0102
-0.0353
-0.0056
0.0046
-0.0019
0.0259
0.0029
-0.0328
0.0186
-0.0096
0.0187
0.0195
-0.0172
0.0164
-0.0028
-0.0324
-0.0226
-0.0174
-0.0148
-0.0269
0.0082
-0.0247
-0.0025
-0.0226
0.0058
-0.00

In [439]:
y_pred = model_ffnn.predict(X_test[:10])
for h in y_pred:
    max_index = np.argmax(h)
    print(max_index)

8
8
8
8
8
8
8
8
8
8


In [440]:
model_ffnn.y.shape

(10, 10)

In [441]:
for h in y_test[:10].flatten():
    print(h)

9
6
5
9
6
9
8
8
1
2


In [442]:
model_ffnn.save(filename="try1.pth")

Model saved to try1.pth


In [443]:
new_model = FFNN()
new_model.load(filename="try1.pth")

Model loaded from try1.pth


In [444]:
y_pred = new_model.predict(X_test[:10])
for h in y_pred:
    max_index = np.argmax(h)
    print(max_index)

8
8
8
8
8
8
8
8
8
8
